In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import json
cwd = os.getcwd()

In [7]:
bestiaryFiles = os.listdir("./bestiary")
spellFiles = os.listdir("./spells")

In [8]:
def has_dice(tag):
    return tag.has_attr("data-packed-dice")

In [127]:
headerPD = ["Name","CR","Size","Armor Class","Hit Points","Speed","Stats","Saving Throws","Skills","Resistances","Damage Immunities","Condition Immunities","Damage Vulnerabilities","Feature Quantity","Feature List","Legendary Resistances","Action List","Multiattack Array","Bonuses","Reactions","Legendary Actions","Lair Actions","Mythic Actions","Region Effects","Spellcaster","nonKeywordedFeature","MaxDamage","StatTotal","dResists","dVuln","dImm","cImm"]
df = pd.DataFrame(columns=headerPD)

In [128]:
conditions = ['blinded','charmed','deafened','frightened','grappled','incapacitated','invisible']
conditions += 'paralyzed','petrified','poisoned','prone','restrained','stunned','exhaustion'
keywordedFeatures = ["Aggressive","Ambusher","Amorphous","Amphibious","Angelic Weapons","Antimagic Susceptibility",\
                    "Avoidance","Blind Senses","Blood Frenzy","Breath Weapon","Brute","Chameleon Skin","Change Shape",\
                    "Charge","Charm","Constrict","Damage Absorption","Damage Transfer","Death Burst","Devil Sight",\
                    "Dive","Echolocation","Elemental Body","Enlarge","Etherealness","False Appearance","Fey Ancestry",\
                     "Fiendish Blessing","Flyby","Frightful Presence","Grappler","Hold Breath","Horrifying Visage",\
                     "Illumination","Illusory Appearance","Immutable Form","Incorporeal Movement","Innate Spellcasting",\
                    "Inscrutable","Invisibility","Keen Senses","Labyrinthine Recall","Leadership","Legendary Resistance",\
                    "Life Drain","Light Sensitivity","Magic Resistance","Magic Weapons","Martial Advantage",\
                    "Mimicry","Nimble Escape","Otherworldly Perception","Pack Tactics","Parry","Possession","Pounce",\
                    "Psychic Defense","Rampage","Reactive","Read Thoughts","Reckless","Redirect Attack","Reel",\
                    "Regeneration","Rejuvenation","Relentless","Shadow Stealth","Shapechanger","Siege Monster",\
                    "Slippery","Spellcasting","Spider Climb","Standing Leap","Steadfast","Stench","Sunlight Sensitivity",\
                    "Superior Invisibility","Sure-Footed","Surprise Attack","Swallow","Teleport","Terrain Camouflage",\
                    "Tunneler","Turn Immunity","Turn Resistance","Two Heads","Undead Fortitude","Web","Web Sense",\
                     "Web Walker","Wounded Fury"]
conditionsRegex = ""
for i, condition in enumerate(conditions):
    condition = "[" + condition[0].upper() + condition[0].lower() + "]" + condition[1:]
    conditionsRegex += condition
    if(i<len(conditions)-1):
        conditionsRegex += "|"
conditionsRegex = re.compile(conditionsRegex)

casters = 0

nums_dict = {
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9',
    'zero': '0'
}

for lengthOfArr, creatureFile in enumerate(bestiaryFiles):
    
    size = ""
    armorClass = 0
    hitPoints = 0
    speed = {"walk":0,
             "climb":0,
             "fly":0,
             "swim":0,
             "burrow":0}
    
    stats = {"strength":0,
             "dexterity":0,
             "constitution":0,
             "intelligence":0,
             "wisdom":0,
             "charisma":0}
    
    savingThrows = {"strength":0,
                    "dexterity":0,
                    "constitution":0,
                    "intelligence":0,
                    "wisdom":0,
                    "charisma":0}
    
    skills = {"Athletics":0,
              "Acrobatics":0,
              "Sleight of Hand,":0,
              "Stealth":0,
              "Arcana":0,
              "History":0,
              "Investigation":0,
              "Nature":0,
              "Religion":0,
              "Animal Handling":0,
              "Insight":0,
              "Medicine":0,
              "Perception":0,
              "Survival":0,
              "Deception":0,
              "Intimidation":0,
              "Performance":0,
              "Persuasion":0,
             }
    multiattackArr = {'name':"N/A"}    

    
    damageResistances = {"Acid":False,
                         "Bludgeoning":False,
                         "Cold":False,
                         "Fire":False,
                         "Force":False,
                         "Lightning":False,
                         "Necrotic":False,
                         "Piercing":False,
                         "Poison":False,
                         "Psychic":False,
                         "Radiant":False,
                         "Slashing":False,
                         "Thunder":False,
    }
    damageVulnerabilities = {"Acid":False,
                         "Bludgeoning":False,
                         "Cold":False,
                         "Fire":False,
                         "Force":False,
                         "Lightning":False,
                         "Necrotic":False,
                         "Piercing":False,
                         "Poison":False,
                         "Psychic":False,
                         "Radiant":False,
                         "Slashing":False,
                         "Thunder":False,
    }
    damageImmunities = {"Acid":False,
                         "Bludgeoning":False,
                         "Cold":False,
                         "Fire":False,
                         "Force":False,
                         "Lightning":False,
                         "Necrotic":False,
                         "Piercing":False,
                         "Poison":False,
                         "Psychic":False,
                         "Radiant":False,
                         "Slashing":False,
                         "Thunder":False,
    }
    
    conditionImmunities = {"blinded":False,
                           "charmed":False,
                           "deafened":False,
                           "exhaustion":False,
                           "frightened":False,
                           "grappled":False,
                           "incapacitated":False,
                           "invisible":False,
                           "paralyzed":False,
                           "petrified":False,
                           "poisoned":False,
                           "prone":False,
                           "restrained":False,
                           "stunned":False,
                           "unconscious":False,
    }
    
    challengeRating = -1
    proficiencyBonus = -1
    
    actions = []
    bonuses = []
    reactions = []
    features = []

    legendaryAct = 0
    legendaryRes = 0
    mythic = False
    regional = False
    lair = False
    bonus = False
    reaction = False
    action = False
    multiAttack = False
    spellCaster = False
    with open(cwd + "\\bestiary\\" + creatureFile, "r") as f:
        soup = BeautifulSoup(f,'html.parser')
        name = soup.find_all("h1", {"class": "stats-name"})[0].string
        placesOfInterest = soup.find_all("td", {"class": "mon__sect-row-inner"})
        headersOfInterest = soup.find_all("h3", {"class":"mon__sect-header-inner"})
        
        data = []
        rows = soup.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele])
            
        armorClass = ''.join(filter(lambda i: i.isdigit(), data[4][0]))[-2:]
        try:
            hitPoints = re.search(r'\d+', data[5][0]).group()
        except:
            hitPoints = 0
        speed["walk"] = re.search(r'\d+', data[6][0]).group()
        if "fly" in data[6][0]:
            speed["fly"] = re.search(r'\d+', data[6][0].split("fly",1)[1]).group()
        if "climb" in data[6][0]:
            speed["climb"] = re.search(r'\d+', data[6][0].split("climb",1)[1]).group()
        if "swim" in data[6][0]:
            speed["swim"] = re.search(r'\d+', data[6][0].split("swim",1)[1]).group()
        if "burrow" in data[6][0]:
            speed["burrow"] = re.search(r'\d+', data[6][0].split("burrow",1)[1]).group()
            
        stats["strength"],savingThrows["strength"] = re.search(r'\d+', data[9][0]).group(), (int(re.search(r'\d+', data[9][0]).group())-10)//2
        skills["Athletics"] = savingThrows["strength"]
        stats["dexterity"], savingThrows["dexterity"] = re.search(r'\d+', data[9][1]).group(), (int(re.search(r'\d+', data[9][1]).group())-10)//2
        skills["Acrobatics"],skills["Sleight of Hand"], skills["Stealth"] = savingThrows["dexterity"],savingThrows["dexterity"],savingThrows["dexterity"]
        stats["constitution"], savingThrows["constitution"] = re.search(r'\d+', data[9][2]).group(), (int(re.search(r'\d+', data[9][2]).group())-10)//2
        stats["intelligence"], savingThrows["intelligence"] = re.search(r'\d+', data[9][3]).group(), (int(re.search(r'\d+', data[9][3]).group())-10)//2
        skills["Arcana"],skills["History"],skills["Investigation"],skills["Nature"],skills["Religion"] = savingThrows["intelligence"], savingThrows["intelligence"], savingThrows["intelligence"], savingThrows["intelligence"], savingThrows["intelligence"]
        stats["wisdom"], savingThrows["wisdom"] = re.search(r'\d+', data[9][4]).group(), (int(re.search(r'\d+', data[9][4]).group())-10)//2
        skills["Animal Handling"],skills["Insight"],skills["Medicine"],skills["Perception"],skills["Survival"] = savingThrows["wisdom"], savingThrows["wisdom"], savingThrows["wisdom"], savingThrows["wisdom"], savingThrows["wisdom"]
        stats["charisma"], savingThrows["charisma"] = re.search(r'\d+', data[9][5]).group(), (int(re.search(r'\d+', data[9][5]).group())-10)//2
        skills["Deception"],skills["Intimidation"],skills["Performance"],skills["Persuasion"] = savingThrows["charisma"], savingThrows["charisma"], savingThrows["charisma"], savingThrows["charisma"]

        try:
            if "Str" in data[11][0]:
                savingThrows["strength"] = re.search(r'\d+', data[11][0].split("Str",1)[1]).group()
        except:
            continue
        try:
            if "Dex" in data[11][0]:
                savingThrows["dexterity"] = re.search(r'\d+', data[11][0].split("Dex",1)[1]).group()
        except:
            continue
        try:
            if "Con" in data[11][0]:
                savingThrows["constitution"] = re.search(r'\d+', data[11][0].split("Con",1)[1]).group()
        except:
            continue
        try:
            if "Int" in data[11][0]:
                savingThrows["intelligence"] = re.search(r'\d+', data[11][0].split("Int",1)[1]).group()
        except:
            continue
        try:
            if "Wis" in data[11][0]:
                savingThrows["wisdom"] = re.search(r'\d+', data[11][0].split("Wis",1)[1]).group()
        except:
            continue
        try:
            if "Cha" in data[11][0]:
                savingThrows["charisma"] = re.search(r'\d+', data[11][0].split("Cha",1)[1]).group()
        except:
            continue
        
        
        for i in data:
            if(len(i) == 2):
                if "Challenge\\n" in i[0]:
                    challengeRating = re.search(r'\d+[\/]?\d?', i[0].split("Challenge\\n",1)[1]).group() 
                
                if "Proficiency Bonus" in i[1]:
                    try:
                        proficiencyBonus = re.search(r'\d+', i[1].split("Proficiency Bonus",1)[1]).group()
                    except:
                        proficiencyBonus = 0
            if i:
                if "Skills" in i[0]:
                    for k in skills:
                        if k in i[0]:
                            skills[k] = re.search(r'\d+', i[0].split(k,1)[1]).group()
                if "Damage Resistances" in i[0]:
                    for k in damageResistances:
                        if k.lower() in i[0]:
                            damageResistances[k] = True
                if "Damage Vulnerabilities" in i[0]:
                    for k in damageVulnerabilities:
                        if k.lower() in i[0]:
                            damageVulnerabilities[k] = True
                if "Damage Immunities" in i[0]:
                    for k in damageImmunities:
                        if k.lower() in i[0]:
                            damageImmunities[k] = True
                if "Condition Immunities" in i[0]:
                    for k in conditionImmunities:
                        if k.lower() in i[0]:
                            conditionImmunities[k] = True
                        
                    
        if "Tiny" in data[2][0]:
            size = "Tiny"
        elif "Small" in data[2][0]:
            size = "Small"
        elif "Medium" in data[2][0]:
            size = "Medium"
        elif "Large" in data[2][0]:
            size = "Large"
        elif "Huge" in data[2][0]:
            size = "Huge"
        elif "Gargantuan" in data[2][0]:
            size = "Gargantuan"
            
#         print("Name:",name)
#         print("Size:",size)
#         print("AC:",armorClass)
#         print("HP:",hitPoints)
#         print("Speed:",speed)
#         print("Stats:",stats)
#         print("Saving Throws:",savingThrows)
#         print("Skills:",skills)
#         print("Resistances:", damageResistances,sep='\n')
#         print("Vulnerabilities:", damageVulnerabilities,sep='\n')
#         print("Immunities:", damageImmunities,sep='\n')
#         print("Condition Immunities:", conditionImmunities,sep='\n')
#         print("CR:",challengeRating)
#         print("PB:",proficiencyBonus)
#         print("")
        
        for header in headersOfInterest:
            if(header.string == "Legendary Actions"):
                instances = soup(text=re.compile("take \d legendary actions"))
                match = re.search(r'take \d legendary actions', instances[0])
                legendaryAct = match.group()[5]
            elif(header.string == "Mythic Actions"):
                mythic = True
            elif(header.string == "Regional Effects"):
                regional = True
            elif(header.string == "Lair Actions"):
                lair = True
            elif(header.string == "Bonus Actions"):
                bonus = True
            elif(header.string == "Reactions"):
                reaction = True
            else:
                action = True
        currentIndexOfInterest = 0
        if(len(placesOfInterest)-len(headersOfInterest)>0):
            featureList = placesOfInterest[currentIndexOfInterest]
            featureList = featureList.find_all("span",{'class': ["rd__h","rd__h--3"]})
            for featureOriginal in featureList:
                featureHTML = featureOriginal.find("span",{'class':'entry-title-inner'})
                feature = featureHTML.string
                if(feature == None):
                    featureHTML.find(True).replaceWith('')
                    featureHTML = featureOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    feature = featureHTML.string
                feature = re.sub("[.]", "", feature)
                feature = re.sub(" \(", "_(", feature)
                feature = feature.replace("\\","")
                featureSplit = feature.split("_")
                if(featureSplit[0]=="Legendary Resistance"):
                    legendaryRes = featureSplit[1][1]
                else:
                    features.append(featureSplit[0])
                if(featureSplit[0]=="Spellcasting" or featureSplit[0]=="Innate Spellcasting"):
                    spellCaster = True
            currentIndexOfInterest += 1

        if(action):
            actionList = placesOfInterest[currentIndexOfInterest]
            actionList = actionList.find_all("div",{'class': ['rd__b',"rd__b--3"]})
            actionNames = []
            for i, actionOriginal in enumerate(actionList):
                actionEntry = {}
                #Get action name
                actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'})
                actionName = actionNameHTML.string
                if(actionName == None):
                    actionNameHTML.find(True).replaceWith('')
                    actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    actionName = actionNameHTML.string
                actionName = re.sub("[.]", "", actionName)
                actionName = re.sub(" \(", "_(", actionName)
                actionName = actionName.replace("\\","")
                actionSplit = actionName.split("_")
                actionName = actionSplit[0]
                if(actionName == "Spellcasting"):
                    spellCaster = True
                elif(actionName == "Multiattack"):
                    multiAttack = True
                    continue
                actionNames.append(actionName)

                actionEntry["name"] = actionName
                
                toHit = actionOriginal.find("span",{"class":"rd__roller--roll-prof-bonus"})
                if(toHit != None):
                    actionEntry['toHit'] = toHit.string
                
                
                importantStats = actionOriginal.find_all(has_dice)
                damageIncrement = 0
                for stat in importantStats:
                    if '"subtype":"damage"' in str(stat) or 'subType&quot;:&quot;damage' in str(stat):
                        damageFormula = stat.string.replace(" ","")
                        diceRolls = re.search(r"[\d]+d[\d]+",damageFormula).group()
                        try:
                            modifier = re.search(r"[+-]\d+",damageFormula).group()
                        except:
                            modifier = 0
                        
                        PB = 0
                        if('PB' in damageFormula):
                            #FIX THIS BEFORE CALLING IT GOOD
                            pass
                        
                        numDice,diceType = diceRolls.split("d")
                        averagePerDie = (1 + int(diceType))/2
                        average = int(averagePerDie * int(numDice))
                        damageTotal = average + int(modifier) + PB

                        damageIncrement += 1
                        actionEntry['damage'+str(damageIncrement)]=damageTotal
                
                typeIncrement = 0
                dtypes = re.findall(r'\) [\S]+ damage',str(actionOriginal))
                for dtype in dtypes:
                    if(dtype == "half" or dtype == "full"):
                        continue
                    typeIncrement += 1
                    actionEntry['dtype' + str(typeIncrement)] = dtype.split(" ")[1]
                
                dc = actionOriginal.find("span",{"class":"rd__dc--rollable-text"})
                if(dc != None and actionName != "Spellcasting"):
                    actionEntry['DC'] = dc.string
                    
                appliedConditions = re.findall(conditionsRegex,str(actionOriginal))
                abilityConditions = []
                for appliedCondition in appliedConditions:
                    if(appliedCondition not in abilityConditions):
                        abilityConditions.append(appliedCondition)
                actionEntry["conditions"] = abilityConditions
                if("Spellcasting" in actionNames):
                    spellCaster = True
                else:
                    actions.append(actionEntry)
            multiAttackArr = {'name':"N/A"}    
            if(multiAttack):
                multiAttackText = actionList[0]
                multiAttackText = multiAttackText.find("p").string
                match = re.search(r"level",str(multiAttackText))
                
                if(match==None):
                    regex = "one|two|three|four|five|six|seven|each"
                    if("Spellcasting" in actionNames):
                        actionNames.pop(actionNames.index("Spellcasting"))

                    for actionName in actionNames:
                        if " " in actionName:
                            newActionNameSplit = actionName.split(" ")
                            regexPiece = ""
                            for part in newActionNameSplit:
                                regexPiece += "[" + part[0].lower() + part[0].upper() + "]" + part[1:]
                                if(newActionNameSplit.index(part)<len(newActionNameSplit)-1):
                                    regexPiece += " "
                        else:
                            regexPiece = "[" + actionName[0].lower() + actionName[0].upper() + "]" + actionName[1:]
                        regex += "|"+ regexPiece
                    matches = re.findall(re.compile(regex),str(multiAttackText))
                    matches = [x.lower() for x in matches]
#                     print(name,matches,regex,actionNames)
                    multiattackArr = {'name':"Multiattack"}
    #                 print(name, matches)
                    if("each" in matches):
                        indexMatch = matches.index("each")
                        numTimes = 1
                        try:
                            if(matches[indexMatch-1] in list(num_dict.keys())):
                                numTimes = int(num_dict[matches[indexMatch-1]])
                        except:
                            pass
                        for actionName in actionNames:
                            multiattackArr[actionName] = numTimes

                    else:
                        for actionName in actionNames:
                            try:
                                indexMatch = matches.index(actionName.lower())
                                
                                if(matches[indexMatch-1] in list(nums_dict.keys())):
                                    multiattackArr[actionName] = int(nums_dict[matches[indexMatch-1]])
                                else:
                                    multiattackArr[actionName] = 1
                            except:
                                pass
    #                 print(name,multiattackArr)
                    if len(multiattackArr.keys())==1:
                        multiattackArr["complicated"] = True
                    else:
                        multiattackArr["complicated"] = False
                    actions.append(multiattackArr)
                    
                    
                
            currentIndexOfInterest += 1

            
        if(bonus):
            actionList = placesOfInterest[currentIndexOfInterest]
            actionList = actionList.find_all("div",{'class': ['rd__b',"rd__b--3"]})
            actionNames = []
            for i, actionOriginal in enumerate(actionList):
                actionEntry = {}
                #Get action name
                actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'})
                actionName = actionNameHTML.string
                if(actionName == None):
                    actionNameHTML.find(True).replaceWith('')
                    actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    actionName = actionNameHTML.string
                actionName = re.sub("[.]", "", actionName)
                actionName = re.sub(" \(", "_(", actionName)
                actionName = actionName.replace("\\","")
                actionSplit = actionName.split("_")
                actionName = actionSplit[0]
                if(actionName == "Spellcasting"):
                    spellCaster = True
                elif(actionName == "Multiattack"):
                    multiAttack = True
                    continue
                actionNames.append(actionName)

                actionEntry["name"] = actionName
                
                toHit = actionOriginal.find("span",{"class":"rd__roller--roll-prof-bonus"})
                if(toHit != None):
                    actionEntry['toHit'] = toHit.string
                
                
                importantStats = actionOriginal.find_all(has_dice)
                damageIncrement = 0
                for stat in importantStats:
                    if '"subtype":"damage"' in str(stat) or 'subType&quot;:&quot;damage' in str(stat):
                        damageFormula = stat.string.replace(" ","")
                        diceRolls = re.search(r"[\d]+d[\d]+",damageFormula).group()
                        try:
                            modifier = re.search(r"[+-]\d+",damageFormula).group()
                        except:
                            modifier = 0
                        
                        PB = 0
                        if('PB' in damageFormula):
                            #FIX THIS BEFORE CALLING IT GOOD
                            pass
                        
                        numDice,diceType = diceRolls.split("d")
                        averagePerDie = (1 + int(diceType))/2
                        average = int(averagePerDie * int(numDice))
                        damageTotal = average + int(modifier) + PB

                        damageIncrement += 1
                        actionEntry['damage'+str(damageIncrement)]=damageTotal
                
                typeIncrement = 0
                dtypes = re.findall(r'\) [\S]+ damage',str(actionOriginal))
                for dtype in dtypes:
                    if(dtype == "half" or dtype == "full"):
                        continue
                    typeIncrement += 1
                    actionEntry['dtype' + str(typeIncrement)] = dtype.split(" ")[1]
                
                dc = actionOriginal.find("span",{"class":"rd__dc--rollable-text"})
                if(dc != None and actionName != "Spellcasting"):
                    actionEntry['DC'] = dc.string
                    
                appliedConditions = re.findall(conditionsRegex,str(actionOriginal))
                abilityConditions = []
                for appliedCondition in appliedConditions:
                    if(appliedCondition not in abilityConditions):
                        abilityConditions.append(appliedCondition)
                actionEntry["conditions"] = abilityConditions
                if("Spellcasting" in actionNames):
                    spellCaster = True
                else:
                    bonuses.append(actionEntry)
            currentIndexOfInterest += 1
        if(reaction):
            actionList = placesOfInterest[currentIndexOfInterest]
            actionList = actionList.find_all("div",{'class': ['rd__b',"rd__b--3"]})
            actionNames = []
            for i, actionOriginal in enumerate(actionList):
                actionEntry = {}
                #Get action name
                actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'})
                actionName = actionNameHTML.string
                if(actionName == None):
                    actionNameHTML.find(True).replaceWith('')
                    actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    actionName = actionNameHTML.string
                actionName = re.sub("[.]", "", actionName)
                actionName = re.sub(" \(", "_(", actionName)
                actionName = actionName.replace("\\","")
                actionSplit = actionName.split("_")
                actionName = actionSplit[0]
                if(actionName == "Spellcasting"):
                    spellCaster = True
                elif(actionName == "Multiattack"):
                    multiAttack = True
                    continue
                actionNames.append(actionName)

                actionEntry["name"] = actionName
                
                toHit = actionOriginal.find("span",{"class":"rd__roller--roll-prof-bonus"})
                if(toHit != None):
                    actionEntry['toHit'] = toHit.string
                
                
                importantStats = actionOriginal.find_all(has_dice)
                damageIncrement = 0
                for stat in importantStats:
                    if '"subtype":"damage"' in str(stat) or 'subType&quot;:&quot;damage' in str(stat):
                        damageFormula = stat.string.replace(" ","")
                        diceRolls = re.search(r"[\d]+d[\d]+",damageFormula).group()
                        try:
                            modifier = re.search(r"[+-]\d+",damageFormula).group()
                        except:
                            modifier = 0
                        
                       
                        
                        numDice,diceType = diceRolls.split("d")
                        averagePerDie = (1 + int(diceType))/2
                        average = int(averagePerDie * int(numDice))
                        
                        if('PB' in damageFormula):
                            damageTotal = average + int(modifier) + proficiencyBonus
                        else:
                            damageTotal = average + int(modifier)

                        damageIncrement += 1
                        actionEntry['damage'+str(damageIncrement)]=damageTotal
                
                typeIncrement = 0
                dtypes = re.findall(r'\) [\S]+ damage',str(actionOriginal))
                for dtype in dtypes:
                    if(dtype == "half" or dtype == "full"):
                        continue
                    typeIncrement += 1
                    actionEntry['dtype' + str(typeIncrement)] = dtype.split(" ")[1]
                
                dc = actionOriginal.find("span",{"class":"rd__dc--rollable-text"})
                if(dc != None and actionName != "Spellcasting"):
                    actionEntry['DC'] = dc.string
                    
                appliedConditions = re.findall(conditionsRegex,str(actionOriginal))
                abilityConditions = []
                for appliedCondition in appliedConditions:
                    if(appliedCondition not in abilityConditions):
                        abilityConditions.append(appliedCondition)
                actionEntry["conditions"] = abilityConditions
                if("Spellcasting" in actionNames):
                    spellCaster = True
                else:
                    reactions.append(actionEntry)
            currentIndexOfInterest += 1
            
        #We don't care about gathering the info in these, though we leave these here just in case
        #we care about trying to access the associated elements of information at a later date
        if(legendaryAct!=0):
            currentIndexOfInterest += 1
        if(mythic):
            currentIndexOfInterest += 1
        if(lair):
            currentIndexOfInterest += 1
        if(regional):
            currentIndexOfInterest += 1
        if(spellCaster):
            casters += 1

    if(not type(challengeRating) == int and "/" in challengeRating):
        challengeRating = challengeRating.split("/")
        challengeRating = float(int(challengeRating[0])/int(challengeRating[1]))

    listToAdd = []
    listToAdd.append(name)
    listToAdd.append(float(challengeRating))
    listToAdd.append(size)
    listToAdd.append(armorClass)
    listToAdd.append(hitPoints)
    listToAdd.append(speed)
    listToAdd.append(stats)
    listToAdd.append(savingThrows)
    listToAdd.append(skills)
    listToAdd.append(damageResistances)
    listToAdd.append(damageImmunities)
    listToAdd.append(conditionImmunities)
    listToAdd.append(damageVulnerabilities)
    listToAdd.append(len(features))
    listToAdd.append(features)
    listToAdd.append(legendaryRes)
    listToAdd.append(actions)
    listToAdd.append(multiattackArr)
    listToAdd.append(bonuses)
    listToAdd.append(reactions)
    listToAdd.append(legendaryAct)
    listToAdd.append(lair)
    listToAdd.append(mythic)
    listToAdd.append(regional)
    listToAdd.append(spellCaster)
    
    featureExpunge = False
    for feature in features:
        if(feature not in keywordedFeatures):
            featureExpunge = True
    if(len(features)==0):
        featureExpunge = True
    listToAdd.append(featureExpunge)
    
    maxDamagePerRound = 0
    maxAlternative = 0
    
    if multiattackArr['name'] != "N/A":
        for key in multiattackArr:
            if(key == "name"):
                continue
            for potentialAction in actions:
                maxDamagePerAttack = 0
                if potentialAction['name'] == key:
                    if('damage1' in potentialAction and 'dtype1' in potentialAction):
                        maxDamagePerAttack = potentialAction['damage1']
                    elif('damage2' in potentialAction and 'dtype2' in potentialAction):
                        maxDamagePerAttack += potentialAction['damage2']
                    maxDamagePerRound += maxDamagePerAttack * multiattackArr[key]
                else:
                    if('damage1' in potentialAction and 'dtype1' in potentialAction):
                        maxDamagePerAttack = potentialAction['damage1']
                    elif('damage2' in potentialAction and 'dtype2' in potentialAction):
                        maxDamagePerAttack += potentialAction['damage2']
                    if(maxDamagePerAttack > maxAlternative):
                        maxAlternative = maxDamagePerAttack
        if(maxDamagePerRound < maxAlternative):
            maxDamagePerRound = maxAlternative
    else:
        for potentialAction in actions:
            maxDamagePerAttack = 0
            if('damage1' in potentialAction and 'dtype1' in potentialAction):
                maxDamagePerAttack = potentialAction['damage1']
            elif('damage2' in potentialAction and 'dtype2' in potentialAction):
                maxDamagePerAttack += potentialAction['damage2']
            if(maxDamagePerAttack > maxDamagePerRound):
                maxDamagePerRound = maxDamagePerAttack
    listToAdd.append(maxDamagePerRound)
    statTotal = 0
    for stat in stats:
        statTotal += int(stats[stat])
    listToAdd.append(statTotal)
        
    dResistBool = False
    for resist in damageResistances:
        if(damageResistances[resist]):
            dResistBool = True
            break
    dVulnBool = False
    for vulns in damageVulnerabilities:
        if(damageVulnerabilities[vulns]):
            dVulnBool = True
            break
            
    dImmBool = False
    for imms in damageImmunities:
        if(damageImmunities[imms]):
            dImmBool = True
            break
    
    cImmBool = False
    for imms in conditionImmunities:
        if(conditionImmunities[imms]):
            cImmBool = True
            break   
    listToAdd.append(dResistBool)
    listToAdd.append(dVulnBool)
    listToAdd.append(dImmBool)
    listToAdd.append(cImmBool)
    
    df.loc[lengthOfArr] = listToAdd

In [129]:
display(df)

,Name,CR,Size,Armor Class,Hit Points,Speed,Stats,Saving Throws,Skills,Resistances,...,Mythic Actions,Region Effects,Spellcaster,nonKeywordedFeature,MaxDamage,StatTotal,dResists,dVuln,dImm,cImm
0,Aarakocra Simulacrum,0.125,Medium,12,6,"{'walk': '20', 'climb': 0, 'fly': '50', 'swim'...","{'strength': '10', 'dexterity': '14', 'constit...","{'strength': 0, 'dexterity': 2, 'constitution'...","{'Athletics': 0, 'Acrobatics': 2, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,5,68,False,False,False,False
1,Aarakocra Spelljammer,6.000,Medium,15,40,"{'walk': '30', 'climb': 0, 'fly': '50', 'swim'...","{'strength': '9', 'dexterity': '14', 'constitu...","{'strength': -1, 'dexterity': 2, 'constitution...","{'Athletics': -1, 'Acrobatics': 2, 'Sleight of...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,True,False,4,74,False,False,False,False
2,Aarakocra,0.250,Medium,12,13,"{'walk': '20', 'climb': 0, 'fly': '50', 'swim'...","{'strength': '10', 'dexterity': '14', 'constit...","{'strength': 0, 'dexterity': 2, 'constitution'...","{'Athletics': 0, 'Acrobatics': 2, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,5,68,False,False,False,False
3,Aartuk Elder,2.000,Large,16,75,"{'walk': '20', 'climb': '20', 'fly': 0, 'swim'...","{'strength': '18', 'dexterity': '10', 'constit...","{'strength': 4, 'dexterity': 0, 'constitution'...","{'Athletics': 4, 'Acrobatics': 0, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,True,False,21,81,False,False,False,False
4,Aartuk Starhorror,2.000,Medium,14,52,"{'walk': '20', 'climb': '20', 'fly': 0, 'swim'...","{'strength': '12', 'dexterity': '10', 'constit...","{'strength': 1, 'dexterity': 0, 'constitution'...","{'Athletics': 1, 'Acrobatics': 0, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,True,False,15,75,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1824,Zombie Clot,6.000,Huge,12,104,"{'walk': '40', 'climb': 0, 'fly': 0, 'swim': 0...","{'strength': '20', 'dexterity': '10', 'constit...","{'strength': 5, 'dexterity': 0, 'constitution'...","{'Athletics': 5, 'Acrobatics': 0, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,36,67,False,False,True,True
1825,Zombie Plague Spreader,4.000,Medium,10,78,"{'walk': '30', 'climb': 0, 'fly': 0, 'swim': 0...","{'strength': '16', 'dexterity': '10', 'constit...","{'strength': 3, 'dexterity': 0, 'constitution'...","{'Athletics': 3, 'Acrobatics': 0, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,14,54,True,False,True,True
1826,Zombie,0.250,Medium,8,22,"{'walk': '20', 'climb': 0, 'fly': 0, 'swim': 0...","{'strength': '13', 'dexterity': '6', 'constitu...","{'strength': 1, 'dexterity': -2, 'constitution...","{'Athletics': 1, 'Acrobatics': -2, 'Sleight of...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,False,4,49,False,False,True,True
1827,Zorbo,0.500,Small,10,27,"{'walk': '30', 'climb': '30', 'fly': 0, 'swim'...","{'strength': '13', 'dexterity': '11', 'constit...","{'strength': 1, 'dexterity': 0, 'constitution'...","{'Athletics': '3', 'Acrobatics': 0, 'Sleight o...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,8,59,False,False,False,False


In [130]:
df = df[(df['Spellcaster'] == False) & (df['CR'] != -1) & (df['nonKeywordedFeature'])]

In [131]:
display(df)

,Name,CR,Size,Armor Class,Hit Points,Speed,Stats,Saving Throws,Skills,Resistances,...,Mythic Actions,Region Effects,Spellcaster,nonKeywordedFeature,MaxDamage,StatTotal,dResists,dVuln,dImm,cImm
0,Aarakocra Simulacrum,0.125,Medium,12,6,"{'walk': '20', 'climb': 0, 'fly': '50', 'swim'...","{'strength': '10', 'dexterity': '14', 'constit...","{'strength': 0, 'dexterity': 2, 'constitution'...","{'Athletics': 0, 'Acrobatics': 2, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,5,68,False,False,False,False
2,Aarakocra,0.250,Medium,12,13,"{'walk': '20', 'climb': 0, 'fly': '50', 'swim'...","{'strength': '10', 'dexterity': '14', 'constit...","{'strength': 0, 'dexterity': 2, 'constitution'...","{'Athletics': 0, 'Acrobatics': 2, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,5,68,False,False,False,False
9,Aboleth Spawn,5.000,Medium,14,93,"{'walk': '30', 'climb': 0, 'fly': 0, 'swim': '...","{'strength': '19', 'dexterity': '11', 'constit...","{'strength': 4, 'dexterity': 0, 'constitution'...","{'Athletics': 4, 'Acrobatics': 0, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,43,76,True,False,False,True
10,Aboleth,10.000,Large,17,135,"{'walk': '10', 'climb': 0, 'fly': 0, 'swim': '...","{'strength': '21', 'dexterity': '9', 'constitu...","{'strength': 5, 'dexterity': -1, 'constitution...","{'Athletics': 5, 'Acrobatics': -1, 'Sleight of...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,True,False,True,36,96,False,False,False,False
11,Abominable Yeti,9.000,Huge,15,137,"{'walk': '40', 'climb': '40', 'fly': 0, 'swim'...","{'strength': '24', 'dexterity': '10', 'constit...","{'strength': 7, 'dexterity': 0, 'constitution'...","{'Athletics': 7, 'Acrobatics': 0, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,49,87,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,Zhent Martial Arts Adept,3.000,Small,16,49,"{'walk': '25', 'climb': 0, 'fly': 0, 'swim': 0...","{'strength': '11', 'dexterity': '17', 'constit...","{'strength': 0, 'dexterity': 3, 'constitution'...","{'Athletics': 0, 'Acrobatics': '5', 'Sleight o...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,36,78,False,False,False,False
1823,Zodar,16.000,Medium,20,200,"{'walk': '30', 'climb': 0, 'fly': '30', 'swim'...","{'strength': '30', 'dexterity': '10', 'constit...","{'strength': 10, 'dexterity': 0, 'constitution...","{'Athletics': 10, 'Acrobatics': 0, 'Sleight of...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,64,111,False,False,True,True
1824,Zombie Clot,6.000,Huge,12,104,"{'walk': '40', 'climb': 0, 'fly': 0, 'swim': 0...","{'strength': '20', 'dexterity': '10', 'constit...","{'strength': 5, 'dexterity': 0, 'constitution'...","{'Athletics': 5, 'Acrobatics': 0, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,36,67,False,False,True,True
1825,Zombie Plague Spreader,4.000,Medium,10,78,"{'walk': '30', 'climb': 0, 'fly': 0, 'swim': 0...","{'strength': '16', 'dexterity': '10', 'constit...","{'strength': 3, 'dexterity': 0, 'constitution'...","{'Athletics': 3, 'Acrobatics': 0, 'Sleight of ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...",...,False,False,False,True,14,54,True,False,True,True


In [132]:
MLdf = df[['Name', 'CR', 'Armor Class','Stats','Saving Throws','Resistances',"Damage Vulnerabilities","Damage Immunities","Condition Immunities",'MaxDamage','Legendary Actions','Legendary Resistances','Feature List']].copy()
vizDF = df[['Name','CR', 'Armor Class','StatTotal',"dResists","dVuln","dImm","cImm",'MaxDamage','Legendary Actions','Legendary Resistances','Feature Quantity']].copy()

In [133]:
display(vizDF)

,Name,CR,Armor Class,StatTotal,dResists,dVuln,dImm,cImm,MaxDamage,Legendary Actions,Legendary Resistances,Feature Quantity
0,Aarakocra Simulacrum,0.125,12,68,False,False,False,False,5,0,0,2
2,Aarakocra,0.250,12,68,False,False,False,False,5,0,0,1
9,Aboleth Spawn,5.000,14,76,True,False,False,True,43,0,0,3
10,Aboleth,10.000,17,96,False,False,False,False,36,3,0,3
11,Abominable Yeti,9.000,15,87,False,False,True,False,49,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1821,Zhent Martial Arts Adept,3.000,16,78,False,False,False,False,36,0,0,3
1823,Zodar,16.000,20,111,False,False,True,True,64,0,3,3
1824,Zombie Clot,6.000,12,67,False,False,True,True,36,0,0,3
1825,Zombie Plague Spreader,4.000,10,54,True,False,True,True,14,0,0,3


In [115]:
display(MLdf)

,Name,CR,Armor Class,Stats,Saving Throws,Resistances,Damage Vulnerabilities,Damage Immunities,Condition Immunities,MaxDamage,Legendary Actions,Legendary Resistances,Feature List
0,Aarakocra Simulacrum,0.125,12,"{'strength': '10', 'dexterity': '14', 'constit...","{'strength': 0, 'dexterity': 2, 'constitution'...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'blinded': False, 'charmed': False, 'deafened...",5,0,0,"[Dive Attack, Simulacra]"
2,Aarakocra,0.250,12,"{'strength': '10', 'dexterity': '14', 'constit...","{'strength': 0, 'dexterity': 2, 'constitution'...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'blinded': False, 'charmed': False, 'deafened...",5,0,0,[Dive Attack]
9,Aboleth Spawn,5.000,14,"{'strength': '19', 'dexterity': '11', 'constit...","{'strength': 4, 'dexterity': 0, 'constitution'...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'blinded': False, 'charmed': True, 'deafened'...",43,0,0,"[Abolethic Vessel, Amphibious, Water Dependency]"
10,Aboleth,10.000,17,"{'strength': '21', 'dexterity': '9', 'constitu...","{'strength': 5, 'dexterity': -1, 'constitution...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'blinded': False, 'charmed': False, 'deafened...",36,3,0,"[Amphibious, Mucous Cloud, Probing Telepathy]"
11,Abominable Yeti,9.000,15,"{'strength': '24', 'dexterity': '10', 'constit...","{'strength': 7, 'dexterity': 0, 'constitution'...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'blinded': False, 'charmed': False, 'deafened...",49,0,0,"[Fear of Fire, Keen Smell, Snow Camouflage]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,Zhent Martial Arts Adept,3.000,16,"{'strength': '11', 'dexterity': '17', 'constit...","{'strength': 0, 'dexterity': 3, 'constitution'...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'blinded': False, 'charmed': False, 'deafened...",36,0,0,"[Brave, Halfling Nimbleness, Unarmored Defense]"
1823,Zodar,16.000,20,"{'strength': '30', 'dexterity': '10', 'constit...","{'strength': 10, 'dexterity': 0, 'constitution...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': True, 'Bludgeoning': False, 'Cold': F...","{'blinded': True, 'charmed': True, 'deafened':...",64,0,3,"[Disembodied Voice, Transport Inhibitor, Unusu..."
1824,Zombie Clot,6.000,12,"{'strength': '20', 'dexterity': '10', 'constit...","{'strength': 5, 'dexterity': 0, 'constitution'...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'blinded': False, 'charmed': True, 'deafened'...",36,0,0,"[Deathly Stench, Undead Fortitude, Unusual Nat..."
1825,Zombie Plague Spreader,4.000,10,"{'strength': '16', 'dexterity': '10', 'constit...","{'strength': 3, 'dexterity': 0, 'constitution'...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'Acid': False, 'Bludgeoning': False, 'Cold': ...","{'blinded': False, 'charmed': True, 'deafened'...",14,0,0,"[Undead Fortitude, Unusual Nature, Viral Aura]"


In [134]:
vizDF.to_csv("vizOutput.csv",index=False)